In [1]:
import torch
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline
)

In [2]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT1EDLpszJ_I53Wdob7rZCqfGHEdLWqtK38WXLeKcssClCa1INsZu0l-P08mjFP2cXs5W0kttvz1wuu/pub?output=csv')

In [3]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

In [4]:
df_ori = df.copy()

In [5]:
df

,text,manual label,cleaned_nostop,fine_tuned_label
0,"Malakan, menurut kalian Danantara lebih banyak...",NaN,malakan danantara peluang risikonya drop penda...,LABEL_0
1,Mumet kondisi skrg,NaN,mumet kondisi skrg,LABEL_2
2,analogi pernikahan dengan papa dana itu agak k...,NaN,analogi pernikahan papa dana kureng si but i g...,LABEL_1
3,"Dalam waktu kedepan tak lama lg, akan muncul b...",NaN,kedepan lg muncul bbrpa perkumpulan d daerah b...,LABEL_0
4,Opportunity apa ya bang?,NaN,opportunity ya bang,LABEL_1
...,...,...,...,...
39639,Asing beli 🚫\nDanantara beli ✅,NaN,asing beli danantara beli,LABEL_1
39640,Ini Endgame of Indonesia..,NaN,endgame of indonesia,LABEL_1
39641,pak prabs ternyata pengikut andry hakim,NaN,prabs pengikut andry hakim,LABEL_1
39642,"Middle class tinggal 30% saja. Menurut NARASI,...",NaN,middle class tinggal narasi rakyat miskin,LABEL_2


In [6]:
df_pred = df.rename(columns={'text':'ori','cleaned_nostop': 'text'})
texts   = df_pred['text'].astype(str).tolist()

# -------------------------------------------------
# 1.  Load the ORIGINAL IndoBERT-large with a fresh head
#     (3 labels so it matches your fine-tuned model)
# -------------------------------------------------
tokenizer_base = AutoTokenizer.from_pretrained("indobenchmark/indobert-large-p1")
model_base     = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-large-p1",
    num_labels=3                 # same number of classes as your fine-tuned model
)

device_id = 0 if torch.cuda.is_available() else -1
pretrained_clf = pipeline(
    task="sentiment-analysis",
    model=model_base,
    tokenizer=tokenizer_base,
    device=device_id
)

# label ↔️ id mapping (random head → default 0,1,2)
id2label = model_base.config.id2label or {0: "LABEL_0", 1: "LABEL_1", 2: "LABEL_2"}

# -------------------------------------------------
# 2.  Predict with safe batching/truncation
# -------------------------------------------------
preds_pretrained = pretrained_clf(
    texts,
    batch_size=32,
    truncation=True,
    max_length=512,
    padding=True
)

# -------------------------------------------------
# 3.  Attach to the same DataFrame
# -------------------------------------------------
df['pretrained_label'] = [p['label'] for p in preds_pretrained]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [7]:
df['fine_tuned_label'].value_counts()

,count
fine_tuned_label,
LABEL_1,19946
LABEL_2,10275
LABEL_0,9423


In [8]:
df['pretrained_label'].value_counts()

,count
pretrained_label,
LABEL_0,32188
LABEL_1,4786
LABEL_2,2670


pada pretrained:

- label_2:

In [9]:
pd.set_option('display.max_colwidth', None)
df.sample(10, random_state =100)

,text,manual label,cleaned_nostop,fine_tuned_label,pretrained_label
17696,"Di indonesia selagi yg korupsi gk di hukum mati maka negara akan sangat susah majunya, harusnya indonesia mencontoh pamerintahan korea utara",NaN,indonesia selagi yg korupsi gk hukum mati negara susah majunya indonesia mencontoh pamerintahan korea utara,LABEL_1,LABEL_0
23223,Dari pada makan gratis yang tidak ada fungsi nya mendingan sekolah gratis,NaN,makan gratis fungsi nya mendingan sekolah gratis,LABEL_1,LABEL_0
35806,"Pejabat2 mental Tikus curut sudah mempersiapkan diri ini, gmn cara mengeruk dan.mempetmainkan dana trilyunan di damantara seperti mengeruk BUMN yg lain",NaN,pejabat mental tikus curut gmn mengeruk danmempetmainkan dana trilyunan damantara mengeruk bumn yg,LABEL_2,LABEL_0
18128,"Mending prabowo dahului focus bayar hutang indonesia tetapi harus penghematan secara ekstrim seperti presiden javier milei presiden argentina,jadi yang pejabat sebagian yg gk penting diberhentikan untuk penghematan anggaran,sebagian pns diberhentikan ,jadi itu lebih bagus 😂",NaN,mending prabowo dahului focus bayar hutang indonesia penghematan ekstrim presiden javier milei presiden argentinajadi pejabat yg gk diberhentikan penghematan anggaransebagian pns diberhentikan bagus,LABEL_1,LABEL_2
31608,kue memang tidak akan lari kemana mana. yakinlah itu....,NaN,kue lari kemana yakinlah,LABEL_1,LABEL_2
21950,"Itu mahasewa bang yg krjaanya cm ngrecokin pemerintahan,kyknya halal deh klw harus ditembak ditempat.",NaN,mahasewa bang yg krjaanya cm ngrecokin pemerintahankyknya halal deh klw ditembak ditempat,LABEL_1,LABEL_0
6628,Walaupun Danantara di duku mereka bertiga ingat krisis kepercayaan akan mebghancurkan nya,NaN,danantara duku bertiga krisis kepercayaan mebghancurkan nya,LABEL_0,LABEL_2
28250,15:00 itu saya bantu jawab ya\nInisial nya ketua B belakang nya UMN😂,NaN,bantu ya inisial nya ketua b nya umn,LABEL_1,LABEL_0
22655,Hati2 pemecah bangsa... Contoh di biayayai antek antek asing,NaN,hati pemecah bangsa contoh biayayai antek antek asing,LABEL_1,LABEL_0
38068,Dagelan muncul... Wkkk,NaN,dagelan muncul wkkk,LABEL_1,LABEL_0


In [10]:
pd.crosstab(df['fine_tuned_label'], df['pretrained_label'], rownames=['Fine-tuned'], colnames=['Pre-trained'])

Pre-trained,LABEL_0,LABEL_1,LABEL_2
Fine-tuned,,,
LABEL_0,7963,780,680
LABEL_1,16133,2327,1486
LABEL_2,8092,1679,504


In [11]:
# Filter rows where the labels are different
df_diff = df[df["fine_tuned_label"] != df["pretrained_label"]][["text", "fine_tuned_label", "pretrained_label"]]

In [12]:
pd.set_option('display.max_colwidth', None)
df_diff.sample(10, random_state =50)

,text,fine_tuned_label,pretrained_label
22971,Rispeeek,LABEL_1,LABEL_0
20464,yang demo kena nasi bungkus singapura,LABEL_1,LABEL_0
10853,Nanti nya Danantara punya Negara atau punya bbrp keluarga yg punya nama di Indonesia,LABEL_1,LABEL_0
13727,bila ada mulyono Siapa lagi yg akan di jadikan pergi saja dulu di danantara seperti yakut...,LABEL_2,LABEL_0
31134,"gila heh korup hampir 1000T tak perlu munafik saya pun apabila ada kesempatan buat korup kemungkinan akan korup juga, tapi ini 980T anjing 980T anjing 980T aku jadi penasaran gaya hidup seperti apa yang mereka lakukan dengan uang 980T?, dugaanku mereka menikmati hasil korup selayaknya bos kartel narkoba yang menghambur hamburkan duit segitu banyaknya hidup dalam jiwa hedonisme cara mendapatkannya melalui atau menggunakan penderitaan rakyat. apakah selama ini rakyat bersedih, tidak!!! mereka happy² aja, lantas seperti apa rasanya hidup dengan kesadaran memiliki pendapatan haknya manusia lain dan tinggal ditengah tengah manusia lain sedangkan penderitaan itu berasal dari manusia modern yang tenggelam kedalam surga yang sebenarnya itu hanyalah ciptaan ilusi pikiran pribadi, dan aku yakin koruptor bisa merasakan yang mana baik dan jahat dilubuk hatinya tetapi tidak tau lagi dan mau di apa jikalau jiwanya sudah tidak ada di dalam raganya.",LABEL_1,LABEL_0
4275,Negara luar punya lembaga seperti ini karena uangnya lagi sehat. Kalo di indonesia?malah terkesan egois dan maksa buat suatu hal yang blum pasti keuntungan nya,LABEL_1,LABEL_0
21715,"mulyono megang danantara , duit di alirin ke ormas , ormas bikin program kudeta , Fufufafa naik jadi presiden YEAY, jenius ini manusia ya",LABEL_1,LABEL_0
22204,Kalau orang itu pintar tetapi tidak jujur maka kepintaran tersebut tidak ada nilainya. Dikutip dari salah satu video pak bennix di filosofi saham fanta.,LABEL_1,LABEL_0
20142,11 ribu teilyun di kantong Jokowi aja hilang mungkin di korupsinya....... ???!,LABEL_2,LABEL_0
1076,Pak Dana mokondo,LABEL_1,LABEL_0


In [13]:
df_diff.to_csv('Perbedaan_FineTuned_PreTrained.csv')

# Pake model lain

w11wo/indonesian-roberta-base-sentiment-classifier

In [14]:
roberta_clf = pipeline(
    "sentiment-analysis",
    model="w11wo/indonesian-roberta-base-sentiment-classifier",
    tokenizer="w11wo/indonesian-roberta-base-sentiment-classifier",
    device=0 if torch.cuda.is_available() else -1
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/808k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
texts = df['cleaned_nostop'].astype(str).tolist()

roberta_preds = roberta_clf(
    texts,
    batch_size=32,
    truncation=True,
    max_length=512,
    padding=True
)

df['w11wo_label'] = [p['label'] for p in roberta_preds]

In [20]:
df['w11wo_label'] = [p['label'] for p in roberta_preds]

In [27]:
#rename, negative: 2, neutral: 1, positive:0
# Define mapping: positive → 0, neutral → 1, negative → 2
sentiment_mapping = {
    'positive': 'LABEL_0',
    'neutral': 'LABEL_1',
    'negative': 'LABEL_2'
}

# Apply mapping to create new column
df['sentiment_label_w'] = df['w11wo_label'].map(sentiment_mapping)

In [29]:
pd.crosstab(df['fine_tuned_label'], df['sentiment_label_w'],
            rownames=['Fine-tuned'], colnames=['w11wo label'])

w11wo label,LABEL_0,LABEL_1,LABEL_2
Fine-tuned,,,
LABEL_0,3373,1837,4213
LABEL_1,3693,5905,10348
LABEL_2,1502,2149,6624


In [30]:
# Filter rows where the labels are different
df_diff_v1 = df[df["fine_tuned_label"] != df["w11wo_label"]][["text", "fine_tuned_label", "w11wo_label"]]
pd.set_option('display.max_colwidth', None)
df_diff_v1.sample(10, random_state =50)

,text,fine_tuned_label,w11wo_label
26232,Knpa pabrik dn PT swasta harus di mangkrak kan.\nYng JD tumbal bnyak rkyat2 nya berhenti bkerja,LABEL_1,negative
21798,Jgn sampai kedepan kita melihat ada yg korupsi 14.000 triliun tanpa divonis 😢,LABEL_0,negative
17585,"Dibandingkan dengan Danantara, lebih baik Indonesia buat suatu wilayah atau Provinsi khusus contohnya seperti Macau atau Hongkong di China. Jika memang hanya Danantara cara memajukan ekonomi Indonesia tolong memilih Pejabat dengan MERITOKRASI 🙏",LABEL_1,neutral
27653,"Mardigu tahu bener FBI CIA katanya pernah ke Markas FBI dan dibangga banggakan , apakah dia pernah jadi anggota CIA ?",LABEL_1,positive
3092,Tapi ternyata si dana dkk scammer,LABEL_2,negative
24895,Hanya orang2 yg tdk setuju dgn danataralah yg yg cari2 kelemahan situasi ini krn selama ini merekalah yg sebenarnya yg korupsi.....hidup pak Prabowo dan jajarannya....,LABEL_0,negative
33855,Rakyat dan bangsa indonesia terlalu gampang di hasut oleh kadrun 😂😂😂,LABEL_1,negative
18617,Ada rumor mau dibelikan BTC😭,LABEL_1,neutral
33062,"Melihatnya sangat membanggakan sekali, tanpa si lampir, si lampir si tukang ngibul pakai nama besar bapaknya untuk mengelabuhi wong cilik.",LABEL_1,negative
35467,Alhamdulillah dewan pengawas dipegang Mulyono 😂,LABEL_0,positive


In [31]:
df_diff_v1.to_csv('PrebedaanFineTunedPreTrained_v1.csv')

In [32]:
df.to_csv('Gabungan.csv')